# Question 5 : Kmeans Clustering  

In [23]:
from pyspark.ml.clustering import KMeans
from pyspark.ml.feature import VectorAssembler
from pyspark.sql import SparkSession 
from pyspark.sql.functions import when, count, col, sum
from pyspark.sql import SparkSession, Row
from pyspark.context import SparkContext, SparkConf
from pyspark.sql.context import SQLContext
from pyspark.sql.session import SparkSession
from pyspark.sql.functions import *
from pyspark.sql.types import IntegerType
import numpy as np
import pandas as pd
import requests
import json

In [20]:
spark = SparkSession.builder \
      .master("local[2]") \
      .appName("SparkByExamples.com") \
      .getOrCreate()

22/12/12 09:28:32 WARN Utils: Your hostname, Sricharraans-Laptop.local resolves to a loopback address: 127.0.0.1; using 172.20.10.2 instead (on interface en0)
22/12/12 09:28:32 WARN Utils: Set SPARK_LOCAL_IP if you need to bind to another address


Setting default log level to "WARN".
To adjust logging level use sc.setLogLevel(newLevel). For SparkR, use setLogLevel(newLevel).


22/12/12 09:28:32 WARN NativeCodeLoader: Unable to load native-hadoop library for your platform... using builtin-java classes where applicable
22/12/12 09:28:32 WARN Utils: Service 'SparkUI' could not bind on port 4040. Attempting port 4041.
22/12/12 09:28:32 WARN Utils: Service 'SparkUI' could not bind on port 4041. Attempting port 4042.


In [21]:
def nearestK(dataPoint, ks):
  closest, ret = np.float("+inf"), 0
  for i in range(len(ks)):
    tDistance = np.sum(np.power((dataPoint-ks[i]), 2))
    if tDistance < closest:
      closest, ret = tDistance, i
  return ret

In this question, we seek to determine the likelihood of citing any black vehicle for illegally parking in the designated street codes. 
In order to identify the nearest cluster to the data point [i.e., data pt] in order to solve this sum, we first build a function called the closest pt. Euclidean distance is used to compute new centres for each cluster. 
The following section of the code chooses the columns Vehicle colour, Street codes 1, 2, and 3 from the data, after which it uses the Vector Assembler function to convert and merge these 4 columns into a single column. 
Then, the k means, km model, and km result parameters are defined.

A list [black list] including all the codes for black vehicles is constructed while keeping in mind the 'Black' vehicle as the inquiry specifies. 
The chance of issuing a ticket to a black vehicle is displayed in an output table along with the prediction, count, and total cars after we create a black list and check if the vehicle colour is on the black list. 
The centres for each cluster formed using k means are then generated. 
Last but not least, we determine the likelihood of issuing a ticket to a black vehicle in street codes 34510, 10030, and 34050 based on all the facts as described before.

In [24]:
#spark = pyspark.sql.SparkSession.builder.appName("ECC-Assignment2-Spark-Kmeans").getOrCreate()

parking = spark.read.format("csv").option("header", "true").load('Parking_Violations_Issued_-_Fiscal_Year_2023.csv').select('Street Code1', 'Street Code2', 'Street Code3', 'Vehicle Color')

parking = parking.select(parking['Street Code1'].cast('float'), parking['Street Code2'].cast('float'), parking['Street Code3'].cast('float'), parking['Vehicle Color'])

vecAssembler = VectorAssembler(inputCols = ["Street Code1", "Street Code2", "Street Code3"], outputCol = "features")
parking = vecAssembler.transform(parking)

kmeans = KMeans(k=5)
KMod = kmeans.fit(parking.select('features'))
res = KMod.transform(parking).cache()
print(res.show())

print("\n\n-x-x-x-x-x-x-x-x-x-x-x-x-x-x-x-x-x-x-x-x-x-x-x-x-x-x-x-x-x-x-x-x-x-x-x-x-x-x-x-x\n\n")

black_list = ['BK', 'BLK', 'BK/', 'BK.', 'BLK.', 'BLAC', 'Black', 'BCK', 'BC', 'B LAC']
BLK_result = res.groupBy('prediction').agg(count(when(col('Vehicle Color').isin(black_list),1)).alias('Count'), count('Vehicle Color').alias('Total_Cars')).orderBy('prediction')
Prob = BLK_result.select('prediction', 'Count', 'Total_Cars', (col('Count') / col ('Total_Cars')).alias('Probability'))
print(Prob.show())

print("\n\n-x-x-x-x-x-x-x-x-x-x-x-x-x-x-x-x-x-x-x-x-x-x-x-x-x-x-x-x-x-x-x-x-x-x-x-x-x-x-x-x\n\n")

ks = np.array(KMod.clusterCenters()).astype(float)
print("The Ks for each clutser is:")
print(ks)

print("\n\n-x-x-x-x-x-x-x-x-x-x-x-x-x-x-x-x-x-x-x-x-x-x-x-x-x-x-x-x-x-x-x-x-x-x-x-x-x-x-x-x\n\n")

cid = nearestK([34510.0, 10030.0, 34050.0], ks)
print('Cluster id for Street Code (34510, 10030, 34050) is:', cid)
print(Prob.filter(col('prediction') == cid).show())
spark.stop()

22/12/12 09:30:00 WARN MemoryStore: Not enough space to cache rdd_131_7 in memory! (computed 19.6 MiB so far)
22/12/12 09:30:00 WARN BlockManager: Persisting block rdd_131_7 to disk instead.
22/12/12 09:30:01 WARN MemoryStore: Not enough space to cache rdd_131_6 in memory! (computed 44.2 MiB so far)
22/12/12 09:30:01 WARN BlockManager: Persisting block rdd_131_6 to disk instead.
22/12/12 09:30:02 WARN MemoryStore: Not enough space to cache rdd_131_6 in memory! (computed 44.2 MiB so far)


22/12/12 09:30:02 WARN MemoryStore: Not enough space to cache rdd_131_7 in memory! (computed 44.2 MiB so far)


22/12/12 09:30:03 WARN MemoryStore: Not enough space to cache rdd_131_7 in memory! (computed 8.4 MiB so far)
22/12/12 09:30:03 WARN MemoryStore: Not enough space to cache rdd_131_6 in memory! (computed 19.6 MiB so far)


22/12/12 09:30:06 WARN MemoryStore: Not enough space to cache rdd_131_7 in memory! (computed 44.2 MiB so far)
22/12/12 09:30:06 WARN MemoryStore: Not enough space to cache rdd_131_7 in memory! (computed 3.7 MiB so far)


22/12/12 09:30:07 WARN MemoryStore: Not enough space to cache rdd_131_0 in memory! (computed 29.5 MiB so far)


...

22/12/12 09:30:38 WARN MemoryStore: Not enough space to cache rdd_197_6 in memory! (computed 57.2 MiB so far)
22/12/12 09:30:38 WARN BlockManager: Persisting block rdd_197_6 to disk instead.
22/12/12 09:30:39 WARN MemoryStore: Not enough space to cache rdd_197_6 in memory! (computed 57.2 MiB so far)


22/12/12 09:30:39 WARN MemoryStore: Not enough space to cache rdd_197_8 in memory! (computed 30.9 MiB so far)
22/12/12 09:30:39 WARN BlockManager: Persisting block rdd_197_8 to disk instead.
22/12/12 09:30:39 WARN MemoryStore: Not enough space to cache rdd_197_8 in memory! (computed 30.9 MiB so far)


22/12/12 09:30:40 WARN MemoryStore: Not enough space to cache rdd_197_7 in memory! (computed 50.6 MiB so far)
22/12/12 09:30:40 WARN BlockManager: Persisting block rdd_197_7 to disk instead.
22/12/12 09:30:40 WARN MemoryStore: Not enough space to cache rdd_197_7 in memory! (computed 50.6 MiB so far)
+----------+------+----------+-------------------+
|prediction| Count|Total_Cars|        Probability|
+----------+------+----------+-------------------+
|         0|119086|    835745|0.14249083153354192|
|         1|686027|   2893714| 0.2370749147980761|
|         2|168959|   1133715|0.14903128211234745|
|         3| 73558|    412634|0.17826451528473175|
|         4| 55139|    311254|0.17715113701350022|
+----------+------+----------+-------------------+

None


-x-x-x-x-x-x-x-x-x-x-x-x-x-x-x-x-x-x-x-x-x-x-x-x-x-x-x-x-x-x-x-x-x-x-x-x-x-x-x-x


The Ks for each clutser is:
[[45217.84711228 12966.7419767  12546.99322683]
 [ 2030.53171405  1074.54772039   723.26006692]
 [16674.91084524 30196.69

/var/folders/qh/sfsll9hn6qx3w2dlh13chvsw0000gn/T/ipykernel_10039/2324378142.py:2: DeprecationWarning: `np.float` is a deprecated alias for the builtin `float`. To silence this warning, use `float` by itself. Doing this will not modify any behavior and is safe. If you specifically wanted the numpy scalar type, use `np.float64` here.
Deprecated in NumPy 1.20; for more details and guidance: https://numpy.org/devdocs/release/1.20.0-notes.html#deprecations
  closest, ret = np.float("+inf"), 0


22/12/12 09:30:40 WARN MemoryStore: Not enough space to cache rdd_197_7 in memory! (computed 25.8 MiB so far)
+----------+------+----------+-------------------+
|prediction| Count|Total_Cars|        Probability|
+----------+------+----------+-------------------+
|         0|119086|    835745|0.14249083153354192|
+----------+------+----------+-------------------+

None


The output for this problem generates four different tables.


The first table shows the features and prediction as a result of the k means model that was fit.


The second table displays the prediction, count, Total_cars and probability of ticketing a black vehicle.


The third table computes the K-centers for each cluster and displays the same.


The last table displays the prediction, count, total_cars and the probability of ticketing a black vehicle in street codes (34510, 10030, 34050). From this table we infer that the cluster id for these street codes is 0. The probability is 0.142 which is greater than 0.004. Hence, we can conclude that the probability is successfully computed.

## References for Kmeans Clustering

1. https://www.scribd.com/document/519217940/lab2-Report

# END OF ASSIGNMENT